In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
# %matplotlib inline
import os

### Read in NDVI data

In [2]:
ds=xr.open_mfdataset(
    '/g/data/oe9/user/rg6346/VI_mask_nc/NDVI/NDVI_????.nc',
    chunks=dict(long=176)) 
ds=ds.rename({'ndvi_evi':'ndvi'})
ds=ds.drop('band')
ds['ndvi']=ds.ndvi/10000;
ds

<xarray.Dataset>
Dimensions:  (lat: 3147, long: 3344, time: 388)
Coordinates:
  * lat      (lat) float64 -24.59 -24.59 -24.6 -24.6 -24.6 -24.61 -24.61 ...
  * long     (long) float64 138.6 138.6 138.6 138.6 138.6 138.6 138.6 138.6 ...
  * time     (time) datetime64[ns] 2000-02-18 2000-03-05 2000-03-21 ...
Data variables:
    ndvi     (time, lat, long) float64 dask.array<shape=(388, 3147, 3344), chunksize=(20, 3147, 176)>

## Subset data for testing

In [3]:
%%time
sub=ds.sel(lat=slice(-30.1, -31),long=slice(148.9,150.6)) ##yarrie
sub.load()

CPU times: user 5.91 s, sys: 38.7 s, total: 44.6 s
Wall time: 3min 29s


### Resample to quarterly, extract seasons

In [4]:
ndvi_q=sub.ndvi.resample(time='Q-FEB').mean(dim='time')
ndvi_jja=ndvi_q.where(ndvi_q['time.season']=='JJA',drop=True)
ndvi_djf=ndvi_q.where(ndvi_q['time.season']=='DJF',drop=True)
ndvi_jja.time

/g/data/oe9/software/anaconda/envs/geo/lib/python3.6/site-packages/xarray/core/common.py:594: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  label=label, base=base)
/g/data/oe9/software/anaconda/envs/geo/lib/python3.6/site-packages/xarray/core/dtypes.py:23: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(dtype, float):
/g/data/oe9/software/anaconda/envs/geo/lib/python3.6/site-packages/xarray/core/dtypes.py:23: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(dtype, float):


<xarray.DataArray 'time' (time: 17)>
array(['2000-08-31T00:00:00.000000000', '2001-08-31T00:00:00.000000000',
       '2002-08-31T00:00:00.000000000', '2003-08-31T00:00:00.000000000',
       '2004-08-31T00:00:00.000000000', '2005-08-31T00:00:00.000000000',
       '2006-08-31T00:00:00.000000000', '2007-08-31T00:00:00.000000000',
       '2008-08-31T00:00:00.000000000', '2009-08-31T00:00:00.000000000',
       '2010-08-31T00:00:00.000000000', '2011-08-31T00:00:00.000000000',
       '2012-08-31T00:00:00.000000000', '2013-08-31T00:00:00.000000000',
       '2014-08-31T00:00:00.000000000', '2015-08-31T00:00:00.000000000',
       '2016-08-31T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2000-08-31 2001-08-31 2002-08-31 ...

In [ ]:
## plot summer in loop

save_path = '/home/156/jbw156/team-drip/figures/djf_ndvi_yarrie/'
for step in ndvi_djf.time:
    fig=plt.figure()
    ndvi_djf.sel(time=step).plot.imshow(robust=True,cmap='Greens',vmin=0,vmax=.8)
#     plt.axes().set_aspect('equal', 'datalim')    
     
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    
    t=pd.to_datetime(step.values) 
    timestring = t.strftime('%Y')
    plt.title('Summer (DJF) '+timestring)
    
    completeName = os.path.join(save_path,"{y}djf_ndvi.png".format(y=timestring))  
    plt.savefig(completeName)
    plt.clf

# Create gif from existing images

In [ ]:
####  Read in images together in a loop

from matplotlib import pyplot as plt  
from matplotlib import animation  
import matplotlib.image as mgimg
import numpy as np

# #set up the figure
fig = plt.figure()
ax = plt.gca()
# ax.set_visible(False)
# fig1 = plt.figure(facecolor='white')
# ax1 = plt.axes(frameon=False)

#initialization of animation, plot array of zeros 
def init():
    imobj.set_data(np.zeros((100, 100)))

    return  imobj,

def animate(i):
    ## Read in picture
#     fname = "heatflow%03d.png" % i 
    fname= '/home/156/jbw156/team-drip/figures/djf_ndvi/2%03ddjf_ndvi.png' % i

    ## here I use [-1::-1], to invert the array
    # IOtherwise it plots up-side down
    img = mgimg.imread(fname)[-1::-1]
    imobj.set_data(img)

    return  imobj,


## create an AxesImage object
imobj = ax.imshow( np.zeros((100, 100)), origin='lower', alpha=1.0, zorder=1, aspect=1 )
plt.axis('off')



anim = animation.FuncAnimation(fig, animate, init_func=init, repeat = True,
                               frames=range(1,17), interval=1000, blit=True, repeat_delay=1000)
anim.save('ndvi_MDB_djf_test.gif')

plt.show()

# Example scripts for animating figs in loop

In [15]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation


fig=plt.figure()
plt.xlabel('Longitude')
plt.ylabel('Latitude')

def make_frame(step):
    ndvi_djf.isel(time=step).plot.imshow(robust=True,cmap='Greens',vmin=0,vmax=.8)
#     plt.axes().set_aspect('equal', 'datalim')    

    
#     t=pd.to_datetime(step.values) 
#     timestring = t.strftime('%Y')
#     plt.title('Summer (DJF) '+timestring)
#         z = np.sin(r2 + 0.5*t) / r2
#     im.set_array(z)
    return ndvi_djf

j=animation.FuncAnimation(fig, make_frame, blit=False)
j.save('ndvitet.gif')

/g/data/oe9/software/anaconda/envs/geo/lib/python3.6/site-packages/matplotlib/animation.py:1218: UserWarning: MovieWriter ffmpeg unavailable
  warnings.warn("MovieWriter %s unavailable" % writer)
/g/data/oe9/software/anaconda/envs/geo/lib/python3.6/site-packages/xarray/plot/utils.py:51: FutureWarning: 'pandas.tseries.converter.register' has been moved and renamed to 'pandas.plotting.register_matplotlib_converters'. 
  converter.register()


KeyboardInterrupt: 

In [14]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

rg = np.arange(-5, 5, 0.1)
x, y = np.meshgrid(rg, rg)
r2 = x**2 + y**2
z = np.sin(r2)/r2

fig = plt.figure()
im = plt.imshow(z)

def make_frame(t):
    z = np.sin(r2 + 0.5*t) / r2
    im.set_array(z)
    return im

j=animation.FuncAnimation(fig, make_frame, blit=False)
j.save('filename2.gif')

/g/data/oe9/software/anaconda/envs/geo/lib/python3.6/site-packages/matplotlib/animation.py:1218: UserWarning: MovieWriter ffmpeg unavailable
  warnings.warn("MovieWriter %s unavailable" % writer)


In [ ]:
"""
A simple example of an animated plot
"""
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots()

x = np.arange(0, 2*np.pi, 0.01)
line, = ax.plot(x, np.sin(x))


def animate(i):
    line.set_ydata(np.sin(x + i/10.0))  # update the data
    return line,


# Init only required for blitting to give a clean slate.
def init():
    line.set_ydata(np.ma.array(x, mask=True))
    return line,

ani = animation.FuncAnimation(fig, animate, np.arange(1, 200), init_func=init,
                              interval=25, blit=True)
plt.show()

In [ ]:
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.animation as animation


def update_line(num, data, line):
    line.set_data(data[..., :num])
    return line,

# Fixing random state for reproducibility
np.random.seed(19680801)


# Set up formatting for the movie files
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)


fig1 = plt.figure()

data = np.random.rand(2, 25)
l, = plt.plot([], [], 'r-')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.xlabel('x')
plt.title('test')
line_ani = animation.FuncAnimation(fig1, update_line, 25, fargs=(data, l),
                                   interval=50, blit=True)
line_ani.save('lines.mp4', writer=writer)

fig2 = plt.figure()

x = np.arange(-9, 10)
y = np.arange(-9, 10).reshape(-1, 1)
base = np.hypot(x, y)
ims = []
for add in np.arange(15):
    ims.append((plt.pcolor(x, y, base + add, norm=plt.Normalize(0, 30)),))

im_ani = animation.ArtistAnimation(fig2, ims, interval=50, repeat_delay=3000,
                                   blit=True)
im_ani.save('im.mp4', writer=writer)

/g/data/oe9/software/anaconda/envs/geo/lib/python3.6/site-packages/matplotlib/animation.py:1218: UserWarning: MovieWriter ffmpeg unavailable
  warnings.warn("MovieWriter %s unavailable" % writer)
